## 1.Import the libraries, functions

In [1]:
import pandas as pd
import numpy as np
import json
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes
import folium
print('Libraries imported.')

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.

Libraries imported.


## 2.Load the List of postal codes of Toronto and transform in Pandas dataframe

In [2]:
data1 = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M',skiprows=1)[0]
data1

,M1A,Not assigned,Not assigned.1
0,M2A,Not assigned,Not assigned
1,M3A,North York,Parkwoods
2,M4A,North York,Victoria Village
3,M5A,Downtown Toronto,"Regent Park, Harbourfront"
4,M6A,North York,"Lawrence Manor, Lawrence Heights"
...,...,...,...
174,M5Z,Not assigned,Not assigned
175,M6Z,Not assigned,Not assigned
176,M7Z,Not assigned,Not assigned
177,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


## 3.Create and clean dataframe 

### Rename the columns like ['PostalCode', 'Borough', 'Neighborhood']

In [3]:
data1=data1.rename({'M1A': 'Postal Code','Not assigned' : 'Borough', 'Not assigned.1':'Neighborhood'}, axis ='columns')
data1

,Postal Code,Borough,Neighborhood
0,M2A,Not assigned,Not assigned
1,M3A,North York,Parkwoods
2,M4A,North York,Victoria Village
3,M5A,Downtown Toronto,"Regent Park, Harbourfront"
4,M6A,North York,"Lawrence Manor, Lawrence Heights"
...,...,...,...
174,M5Z,Not assigned,Not assigned
175,M6Z,Not assigned,Not assigned
176,M7Z,Not assigned,Not assigned
177,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


In [4]:
data1.shape

(179, 3)

In [5]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(data1['Borough'].unique()),
        data1.shape[0]
    )
)

The dataframe has 11 boroughs and 179 neighborhoods.


### Ignore cells with a Borough that is Not assigned

In [6]:
#filter my df with loc.The "~" means we want the opposit of the condition. 
#The ":" means we want to keep all the columns
data1_new=data1.loc[~(data1['Borough']=='Not assigned'), :].reset_index(drop=True)
data1_new

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


### Check if Neighborhood are null

In [7]:
data1_new[data1_new['Neighborhood'].isnull()]

,Postal Code,Borough,Neighborhood


In [8]:
data1_new.shape

(103, 3)

### Update Not assigned Neighborhood with value of Borough

In [9]:
data1_new['Neighborhood']=np.where(data1_new['Neighborhood']=='Not assigned',data1_new['Borough'],data1_new['Neighborhood'])
data1_new        


,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [10]:
data1_new.shape

(103, 3)

In [11]:
!conda install -c conda-forge geocoder --yes
import geocoder # import geocoder

Solving environment: ...working... done

# All requested packages already installed.



In [12]:
# initialize your variable to None
lat_lng_coords = None

In [13]:
# loop until you get the coordinates
while(lat_lng_coords is None):
  g = geocoder.google('{}, Toronto, Ontario'.format('M5G'))
  lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]

KeyboardInterrupt: 

### Geocoder is not responding, so I will use the Geospatial file

In [14]:
geo_df=pd.read_csv("https://cocl.us/Geospatial_data")
geo_df

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


### Merge geodata and data1_new into a new dataframe

In [15]:
loc_Toronto=pd.merge(left=data1_new, right=geo_df, left_on='Postal Code', right_on='Postal Code')

In [16]:
loc_Toronto

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


## Explore and cluster the neighborhoods in Toronto. 
### I will generate maps to visualize the neighborhoods and how they cluster together, in Toronto

In [17]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(loc_Toronto['Borough'].unique()),
        loc_Toronto.shape[0]
    )
)

The dataframe has 10 boroughs and 103 neighborhoods.


In [18]:
Latitude=loc_Toronto.loc[0, 'Latitude']
Longitude=loc_Toronto.loc[0,'Longitude']
print(Latitude, Longitude)

43.7532586 -79.3296565


### Create map of Toronto using latitude and longitude values

In [19]:
map_toronto = folium.Map(location=[Latitude, Longitude], zoom_start=10)


In [20]:
# add markers to map
for lat, lng, borough, neighborhood in zip(loc_Toronto['Latitude'], loc_Toronto['Longitude'], loc_Toronto['Borough'], loc_Toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### We will work just with boroughts that contain the word Toronto. 
### We will generate maps to visualize the neighborhoods and how they cluster together.

In [21]:
to_data=loc_Toronto[loc_Toronto['Borough'].str.contains('Toronto')].reset_index(drop=True)
to_data.head()


,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


### Define Foursquare Credentials and Version

In [22]:
CLIENT_ID = 'A2T30WL1CW2EKPIUBZILXATZMN3VCS02DGGHJOAQNR2HKYMV' # my Foursquare ID
CLIENT_SECRET = '4TGYNOQKWWWGBA5HCN2J5YMIWMGS5W4D2JDALDSNK0J5KHPK' # my Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('My credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

My credentails:
CLIENT_ID: A2T30WL1CW2EKPIUBZILXATZMN3VCS02DGGHJOAQNR2HKYMV
CLIENT_SECRET:4TGYNOQKWWWGBA5HCN2J5YMIWMGS5W4D2JDALDSNK0J5KHPK


### Check the size of the resulting dataframe

In [23]:
print(to_data.shape)
to_data.head()

(39, 5)


,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


### Let's find out how many unique categories can be curated from all the returned Borough

In [24]:
print('There are {} uniques Borough.'.format(len(to_data['Borough'].unique())))

There are 4 uniques Borough.


### Explore Neighborhoods in Toronto

In [25]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [26]:
toronto_venues =getNearbyVenues(to_data['Neighborhood'],to_data['Latitude'],to_data['Longitude'])

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
R

### We write the code to run the above function on each neighborhood and create a new dataframe called toronto_venues.

In [27]:
toronto_venues

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.654260,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
1,"Regent Park, Harbourfront",43.654260,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
2,"Regent Park, Harbourfront",43.654260,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.654260,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront",43.654260,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant
...,...,...,...,...,...,...,...
1585,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,The Ten Spot,43.664815,-79.324213,Spa
1586,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,Toronto Yoga Mamas,43.664824,-79.324335,Yoga Studio
1587,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,Olliffe On Queen,43.664503,-79.324768,Butcher
1588,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,TTC Stop #03049,43.664470,-79.325145,Light Rail Station


### We check how many venues were returned for each neighborhood

In [28]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,56,56,56,56,56,56
"Brockton, Parkdale Village, Exhibition Place",22,22,22,22,22,22
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",17,17,17,17,17,17
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",15,15,15,15,15,15
Central Bay Street,60,60,60,60,60,60
Christie,15,15,15,15,15,15
Church and Wellesley,78,78,78,78,78,78
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,33,33,33,33,33,33


### We find out how many unique categories can be curated from all the returned venues

In [29]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 228 uniques categories.


## 4. Analyze Each Neighborhood

### We check if the Venue Category=Neighborhood exists

In [30]:
pd.set_option('display.max_rows', 10)
toronto_venues[toronto_venues['Venue Category'].str.contains('Nei')].reset_index(drop=True)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
1,"Richmond, Adelaide, King",43.650571,-79.384568,Downtown Toronto,43.653232,-79.385296,Neighborhood
2,"Harbourfront East, Union Station, Toronto Islands",43.640816,-79.381752,Harbourfront,43.639526,-79.380688,Neighborhood
3,Studio District,43.659526,-79.340923,Leslieville,43.662070,-79.337856,Neighborhood


## There is a Venue Category that is Neighborhood, so we will rename that category to Neighborhood Category 

In [31]:
toronto_venues['Venue Category']=np.where(toronto_venues['Venue Category']=='Neighborhood','Neighborhood Category',toronto_venues['Venue Category'])
  

### Analyse each Neighborhood

In [32]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")


# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 


# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()


,Neighborhood,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### We will examine the new dataframe size.

In [33]:
toronto_onehot.shape

(1590, 229)

### Let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [34]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,Berczy Park,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,...,0.0,0.00000,0.0,0.00000,0.00,0.017857,0.000000,0.0,0.000000,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,...,0.0,0.00000,0.0,0.00000,0.00,0.000000,0.000000,0.0,0.000000,0.000000
2,"Business reply mail Processing Centre, South C...",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,...,0.0,0.00000,0.0,0.00000,0.00,0.000000,0.000000,0.0,0.000000,0.058824
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.0,0.066667,0.066667,0.066667,0.133333,0.066667,0.133333,0.00000,0.0,...,0.0,0.00000,0.0,0.00000,0.00,0.000000,0.000000,0.0,0.000000,0.000000
4,Central Bay Street,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,...,0.0,0.00000,0.0,0.00000,0.00,0.016667,0.000000,0.0,0.016667,0.016667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34,"The Annex, North Midtown, Yorkville",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,...,0.0,0.00000,0.0,0.00000,0.00,0.000000,0.000000,0.0,0.000000,0.000000
35,The Beaches,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,...,0.0,0.00000,0.0,0.25000,0.00,0.000000,0.000000,0.0,0.000000,0.000000
36,"The Danforth West, Riverdale",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.02381,0.0,...,0.0,0.02381,0.0,0.02381,0.00,0.000000,0.000000,0.0,0.000000,0.023810
37,"Toronto Dominion Centre, Design Exchange",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.03000,0.0,...,0.0,0.00000,0.0,0.00000,0.01,0.010000,0.000000,0.0,0.010000,0.000000


### Let's confirm the new size

In [35]:
toronto_grouped.shape

(39, 229)

### Let's take a look at all the venue of neighborhood 'Berczy Park'

In [36]:
hood0='Berczy Park'
t=toronto_grouped[toronto_grouped['Neighborhood'] == hood0].T.reset_index()
t.columns= ['venue','freq']
t = t.iloc[1:]
t['freq'] = t['freq'].astype(float)
t = t.round({'freq': 2})
t.sort_values('freq', ascending=False)

,venue,freq
54,Coffee Shop,0.09
53,Cocktail Bar,0.05
47,Cheese Shop,0.04
85,Farmers Market,0.04
20,Bakery,0.04
...,...,...
92,Food Court,0.00
93,Food Truck,0.00
96,Fried Chicken Joint,0.00
97,Frozen Yogurt Shop,0.00


### Now, let's print each neighborhood along with the top 5 most common venues

In [37]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
            venue  freq
0     Coffee Shop  0.09
1    Cocktail Bar  0.05
2     Cheese Shop  0.04
3  Farmers Market  0.04
4          Bakery  0.04


----Brockton, Parkdale Village, Exhibition Place----
            venue  freq
0            Café  0.14
1  Breakfast Spot  0.09
2     Coffee Shop  0.09
3    Climbing Gym  0.05
4         Stadium  0.05


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
                  venue  freq
0    Light Rail Station  0.12
1  Gym / Fitness Center  0.06
2         Auto Workshop  0.06
3                   Spa  0.06
4            Skate Park  0.06


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                venue  freq
0      Airport Lounge  0.13
1    Airport Terminal  0.13
2            Boutique  0.07
3  Airport Food Court  0.07
4        Airport Gate  0.07


----Central Bay Street----
                venue  freq
0         Coffee Sho

### Let's put that into a pandas dataframe

### First, let's write a function to sort the venues in descending order.

In [38]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [39]:
#Return most of common venues for toronto_grouped dataframe
return_most_common_venues(toronto_grouped.iloc[0, :], 10)

array(['Coffee Shop', 'Cocktail Bar', 'Bakery', 'Beer Bar',
       'Farmers Market', 'Seafood Restaurant', 'Restaurant',
       'Cheese Shop', 'Clothing Store', 'Juice Bar'], dtype=object)

### Create the new dataframe and display the top 10 venues for each neighborhood

In [40]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Beer Bar,Farmers Market,Seafood Restaurant,Restaurant,Cheese Shop,Clothing Store,Juice Bar
1,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Breakfast Spot,Gym,Stadium,Burrito Place,Restaurant,Climbing Gym,Pet Store,Bakery
2,"Business reply mail Processing Centre, South C...",Light Rail Station,Yoga Studio,Farmers Market,Burrito Place,Auto Workshop,Spa,Fast Food Restaurant,Garden,Butcher,Garden Center
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Terminal,Boat or Ferry,Harbor / Marina,Boutique,Rental Car Location,Bar,Coffee Shop,Sculpture Garden,Airport Gate
4,Central Bay Street,Coffee Shop,Sandwich Place,Italian Restaurant,Café,Burger Joint,Salad Place,Bubble Tea Shop,Yoga Studio,Ice Cream Shop,Indian Restaurant


## 5. Cluster Neighborhoods

### Run k-means to cluster the neighborhood into 5 clusters.

In [41]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_ 

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 4, 1, 2, 1,
       1, 1, 1, 1, 2, 3, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1])

In [42]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
neighborhoods_venues_sorted

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Beer Bar,Farmers Market,Seafood Restaurant,Restaurant,Cheese Shop,Clothing Store,Juice Bar
1,1,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Breakfast Spot,Gym,Stadium,Burrito Place,Restaurant,Climbing Gym,Pet Store,Bakery
2,1,"Business reply mail Processing Centre, South C...",Light Rail Station,Yoga Studio,Farmers Market,Burrito Place,Auto Workshop,Spa,Fast Food Restaurant,Garden,Butcher,Garden Center
3,1,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Terminal,Boat or Ferry,Harbor / Marina,Boutique,Rental Car Location,Bar,Coffee Shop,Sculpture Garden,Airport Gate
4,1,Central Bay Street,Coffee Shop,Sandwich Place,Italian Restaurant,Café,Burger Joint,Salad Place,Bubble Tea Shop,Yoga Studio,Ice Cream Shop,Indian Restaurant
...,...,...,...,...,...,...,...,...,...,...,...,...
34,1,"The Annex, North Midtown, Yorkville",Coffee Shop,Sandwich Place,Café,History Museum,Burger Joint,Middle Eastern Restaurant,Indian Restaurant,Pub,BBQ Joint,Donut Shop
35,0,The Beaches,Health Food Store,Pub,Neighborhood Category,Trail,Dog Run,Dessert Shop,Diner,Discount Store,Distribution Center,Yoga Studio
36,1,"The Danforth West, Riverdale",Greek Restaurant,Coffee Shop,Italian Restaurant,Furniture / Home Store,Restaurant,Ice Cream Shop,Cosmetics Shop,Brewery,Bubble Tea Shop,Café
37,1,"Toronto Dominion Centre, Design Exchange",Coffee Shop,Hotel,Café,Salad Place,Italian Restaurant,Restaurant,Seafood Restaurant,American Restaurant,Japanese Restaurant,Bakery


### Create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood

In [43]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged=pd.merge(left=loc_Toronto, right=neighborhoods_venues_sorted, left_on='Neighborhood', right_on='Neighborhood')
toronto_merged.head() # check the last columns!


,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1,Coffee Shop,Bakery,Pub,Park,Breakfast Spot,Café,Theater,Yoga Studio,Cosmetics Shop,Shoe Store
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,1,Coffee Shop,Sushi Restaurant,Yoga Studio,Distribution Center,Portuguese Restaurant,Park,Mexican Restaurant,Japanese Restaurant,Italian Restaurant,Fried Chicken Joint
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,1,Coffee Shop,Clothing Store,Italian Restaurant,Café,Cosmetics Shop,Middle Eastern Restaurant,Bubble Tea Shop,Japanese Restaurant,Movie Theater,Hotel
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,1,Coffee Shop,Café,Cocktail Bar,American Restaurant,Gastropub,Gym,Restaurant,Moroccan Restaurant,Cosmetics Shop,Creperie
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Health Food Store,Pub,Neighborhood Category,Trail,Dog Run,Dessert Shop,Diner,Discount Store,Distribution Center,Yoga Studio


### Finally, let's visualize the resulting clusters

In [44]:
# create map
map_clusters = folium.Map(location=[Latitude, Longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 6. Examine Clusters

### Cluster1

In [65]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1]+[2] + list(range(6, toronto_merged.shape[1]))]]

,Borough,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,East Toronto,The Beaches,Health Food Store,Pub,Neighborhood Category,Trail,Dog Run,Dessert Shop,Diner,Discount Store,Distribution Center,Yoga Studio


### Cluster2

In [66]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1]+[2] + list(range(6, toronto_merged.shape[1]))]]

,Borough,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,"Regent Park, Harbourfront",Coffee Shop,Bakery,Pub,Park,Breakfast Spot,Café,Theater,Yoga Studio,Cosmetics Shop,Shoe Store
1,Downtown Toronto,"Queen's Park, Ontario Provincial Government",Coffee Shop,Sushi Restaurant,Yoga Studio,Distribution Center,Portuguese Restaurant,Park,Mexican Restaurant,Japanese Restaurant,Italian Restaurant,Fried Chicken Joint
2,Downtown Toronto,"Garden District, Ryerson",Coffee Shop,Clothing Store,Italian Restaurant,Café,Cosmetics Shop,Middle Eastern Restaurant,Bubble Tea Shop,Japanese Restaurant,Movie Theater,Hotel
3,Downtown Toronto,St. James Town,Coffee Shop,Café,Cocktail Bar,American Restaurant,Gastropub,Gym,Restaurant,Moroccan Restaurant,Cosmetics Shop,Creperie
5,Downtown Toronto,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Beer Bar,Farmers Market,Seafood Restaurant,Restaurant,Cheese Shop,Clothing Store,Juice Bar
...,...,...,...,...,...,...,...,...,...,...,...,...
34,Downtown Toronto,Stn A PO Boxes,Coffee Shop,Seafood Restaurant,Cocktail Bar,Hotel,Italian Restaurant,Japanese Restaurant,Beer Bar,Restaurant,Café,Gym
35,Downtown Toronto,"St. James Town, Cabbagetown",Coffee Shop,Bakery,Park,Café,Pizza Place,Italian Restaurant,Pub,Restaurant,Snack Place,Bar
36,Downtown Toronto,"First Canadian Place, Underground city",Coffee Shop,Café,Hotel,Gym,Japanese Restaurant,Restaurant,Asian Restaurant,Seafood Restaurant,Steakhouse,Salad Place
37,Downtown Toronto,Church and Wellesley,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Gay Bar,Restaurant,Yoga Studio,Pub,Men's Store,Mediterranean Restaurant,Hotel


### Cluster3

In [67]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1]+[2] + list(range(6, toronto_merged.shape[1]))]]

,Borough,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,Central Toronto,"Forest Hill North & West, Forest Hill Road Park",Park,Jewelry Store,Trail,Sushi Restaurant,Department Store,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
29,Central Toronto,"Moore Park, Summerhill East",Gym,Trail,Restaurant,Park,German Restaurant,Gluten-free Restaurant,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
33,Downtown Toronto,Rosedale,Park,Playground,Trail,Deli / Bodega,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run


### Cluster4

In [68]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1]+[2] + list(range(6, toronto_merged.shape[1]))]]

,Borough,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Central Toronto,Roselawn,Home Service,Garden,Yoga Studio,Department Store,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant


### Cluster5

In [69]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1]+[2] + list(range(6, toronto_merged.shape[1]))]]

,Borough,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Central Toronto,Lawrence Park,Park,Swim School,Bus Line,Dessert Shop,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant


In cluster1, we can see that the most common venue is "Health Food Store" and is locateted in East Toronto. In cluster2, the most common venue is "Coffe Shops" and most of them are locateted in Toronto downtown. In cluster3, the Borough belong to this cluster come from Central Toronto and Downtown Toronto downtown and Park, Gym are the most commun venue. In cluster4, the most common venue is "Home Service" and is located in Central Toronto. In cluster5, comes from borough that mostly situated in Central Toronto and the common venue is "Park".
As we can see, examining each cluster, there are discriminating venue categories that distinguish each cluster. 
